## 💻 UnpackAI DL201 Bootcamp - Week 3 - NLP pipelines

### 📕 Learning Objectives

* Getting working examples able to achieve the main NLP tasks
* Knowing the existence of Hugging Face and the strenth of its pre-trained models and all-in-one pipelines

### 📖 Concepts map

* Pipeline

In [70]:
# import (use not verbose mode : ex "import -Uqq pandas as pd" if you are sure that there is no dependency error)

from transformers import pipeline
import pandas as pd
import numpy as np

# Part 1. Introduction

Giving working example able to inspire you to build your own AI project

Hugging Face made available all-in-one ***pipelines*** including all the main steps of NLP.
https://huggingface.co/course/chapter2/2?fw=pt
* choosing a pre-trained model
* adapting the input text into this model (tokenization, vectorization) 
* running the model on the transformed input data
* adapting the model answer to human beings (ex : de-tokenization, to get an output text from an output vector or numbers)

Once the pipeline works, you can decide to tune it, more and more, little by little, as one would do to transform their car for a speed race.
So, you can decide to :
* fine tune the model or train it from scratch (instead of using pre-trained model)
* using a tokenizer from your own (instead of the default one)
* clean the training data before feeding the model


# Part 2. Example of question answering

In [26]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [27]:
my_answer = question_answerer(
    question="Where do I work?",
    context="My name is John and I work at unpackAI in Beijing."
)

In [28]:
my_answer

{'score': 0.5068178772926331, 'start': 30, 'end': 38, 'answer': 'unpackAI'}

In [29]:
my_answer['answer']

'unpackAI'

In [30]:
def my_question_answerer(my_question, my_context):
        
    complete_answer = question_answerer(
        question=my_question,
        context=my_context
    )
    return complete_answer['answer']

In [31]:
context = 'My name is James and I work in Shenzhen'
question = 'Where do you work?'

In [32]:
my_question_answerer(question,context)

'Shenzhen'

# Part 3. Example of Sentiment Analysis

Here, let's apply our pipeline on a list series (i.e. one column of a dataframe) containing text data.

In [35]:
Text_series = pd.Series([ "I've been waiting for a HuggingFace course my whole life.","I hate this so much!"])

In [36]:
Text_series.shape

(2,)

Now, we have to transform this series into a normal list, because pipelines can deal with list but not series.

In [37]:
Text_list = Text_series.to_list()

In [38]:
len(Text_list)

2

We load the sentiment analysis pipeline

In [40]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [63]:
list_of_answers = classifier (Text_list)
list_of_answers

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [42]:
len(list_of_answers)

2

Let's try to get the list of labels, or the list of score

In [61]:
#list_of_answers[:]['label'] # error

In [ ]:
So, to get them, we have to transform it back to a series

In [55]:
series_of_answers = pd.Series(list_of_answers)
series_of_answers.shape

(2,)

In [66]:
series_of_labels = series_of_answers.apply(lambda x: x['label'])
series_of_labels

0    POSITIVE
1    NEGATIVE
dtype: object

In [68]:
series_of_scores = series_of_answers.apply(lambda x: x['score'])
series_of_scores

0    0.959805
1    0.999456
dtype: float64

Here, we remark than we cannot tell if it is positive or not by reading the score.
So, we have to combine both the score and the label to get a signed score.

In [74]:
series_of_signed_scores = np.where(series_of_labels == 'POSITIVE', series_of_scores, -1 * series_of_scores)
series_of_signed_scores

array([ 0.95980495, -0.99945587])

# Part 4. Example of Text Generation

In [ ]:
generator = pipeline("text-generation")

In [ ]:
generator(
    "In this course, we will teach you how to utilize NLP",
    max_length=30,
    num_return_sequences=2
)

# Part 5. Example of Named Entity Recognition (NER)

Named Entity Recognition (NER) is the task of identifying and categorizing key information (entities) in text. An entity can be any word or series of words that consistently refers to the same thing. Examples could be entities such as person (PER), organization (ORG), date (DATE), location (LOC), or more.

In [ ]:
ner_pipeline = pipeline("ner", grouped_entities=True)

In [ ]:
ner_pipeline("My name is John and I work at unpackAI in Beijing.")

# Part 6. How to fine tune a transformer ?

Finetuning a transformer is not hard, but transformers were made to work with shared data, and shared data must respect some strict rules to adapt different models. So, a new data type was invented : the ***Dataset***

And the main difficulty of transformers fine tuning, is to transform our training data into a Dataset.

You may see an example here :
https://www.kaggle.com/code/philanoe/intent-classifier-training